# Reverse Game of Life - Z3 Constraint Satisfaction

[Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) is an example of 2D cellular automata. 

I have previously written an interactive playable demo of the forward version of this game:
- https://life.jamesmcguigan.com/

Using the classic ruleset on a 25x25 board with wraparound, the game evolves at each timestep according to the following rules
- Overpopulation: if a living cell is surrounded by more than three living cells, it dies.
- Stasis: if a living cell is surrounded by two or three living cells, it survives.
- Underpopulation: if a living cell is surrounded by fewer than two living cells, it dies.
- Reproduction: if a dead cell is surrounded by exactly three cells, it becomes a live cell.

Or expressed algebraicly:
- living + 4-8 neighbours = dies
- living + 2-3 neighbours = lives
- living + 0-1 neighbour  = dies
- dead   +   3 neighbours = lives


To reverse the arrow of time:
- any living cell must have had living 2-3 neighbours in the previous timestep
- any dead cell must have had either 0-1 or 4-8 neighbours in the previous timestep
- any dead cell with distance of greater than 2 from a living cell can be ignored and assumed to have 0 neighbours
  - there are a near infinite number of self-contained patterns could have been born and died out in empty space
  - however for the sake of the competition, ignoring them will greatly reduce the search space


Whilst there have been many proposed solutions involving CNN neural networks (when all you have is a hammer, everything looks like a nail), this is in fact a classic constraint satisfaction problem. Here are some previous examples of using the Z3 library 
- https://www.kaggle.com/jamesmcguigan/z3-sudoku-solver
- https://www.kaggle.com/jamesmcguigan/cryptarithmetic-solver

# Z3 solver

Here we define a Z3 solver, with a 3d (= 2d + time) grid of t_cells, representing the current board state and previous timesteps for the required delta to be solved.

As we are going backwards, we can ignore the rules for dead cells, and simply define the constraints required for the past timestep to produce living cells in the current timestep. We also define a rule to ignore any cells that currently have 0 neighbours to reduce the search space and prevent zero-point energy solutions.

NOTE: With Z3 using boolean datatypes is 2-4x faster than using integer constraints

In [ ]:
# TODO: add z3-solver to kaggle-docker image
! python3 -m pip install -q z3-solver
! apt-get install -qq tree moreutils

In [ ]:
# Download git repository and copy to local directory
!rm -rf /ai-games/
!git clone https://github.com/JamesMcGuigan/ai-games/ /ai-games/
# !cd /ai-games/; git checkout ad2f8cc94865f1be6083ca699d4b62b0cc039435
!cp -rf /ai-games/puzzles/game_of_life/* ./   # copy code to kaggle notebook
!rm -rf /kaggle/working/neural_networks/      # not relevant to this notebook
!cd /ai-games/; git log -n1 

Show the new filesystem contents of the notebook

In [ ]:
! pwd
! tree -F

# Dataset Reimport Loop

There are 10 forks of this notebook running in parallel as a poor man's version of cluster compute using a modulo loop to subdivide the data, with the `submission.csv` file acting as a filesystem cache. They each reimport the output of each other, with the accumilated results of multiple 9-hour commit sessions merged together. 

Even still, this is a massive compute job and its going to take a while to finish all 50,000 entries in the test dataset given that each notebook can only compute a few hundred entries per commit cycle.

We also import any boards solved via the Hashmap Solver, which identifies geometric duplicates discovered from the training dataset, and generated data using forward play 
- https://www.kaggle.com/jamesmcguigan/game-of-life-hashmap-solver/

In [ ]:
# Merge submission files from various sources into a single file. Reverse sort puts non-zero entries first, then use awk to deduplicate on id
!find ./ ../input/ /ai-games/puzzles/game_of_life/ -name 'submission.csv' | xargs cat | sort -nr | uniq | awk -F',' '!a[$1]++' | sort -n > ./submission_previous.csv
!find ./ ../input/ /ai-games/puzzles/game_of_life/ -name 'submission.csv' | xargs cat | sort -nr | uniq | awk -F',' '!a[$1]++' | sort -n > ./submission.csv
!find ./ ../input/ /ai-games/puzzles/game_of_life/ -name 'timeouts.csv'   | xargs cat | sort -nr | uniq | awk -F',' '!a[$1]++' | sort -n > ./timeouts.csv

# Count number of non-zero entries in each submission.csv file
!( for FILE in $(find ./ ../input/ /ai-games/puzzles/game_of_life/ -name '*submission.csv' | sort ); do cat $FILE | grep ',1' | wc -l | tr '\n' ' '; echo $FILE; done) | sort -n;

# BUGFIX: previous version of the code was computing to delta=-1, so replay submission.csv forward one step if required and validate we have the correct delta
# This also generates stats
!PYTHONPATH='.' python3 ./constraint_satisfaction/fix_submission.py

# Codebase

Full codebase can be seen either in the output files section of the notebook or on my github repo:
- https://github.com/JamesMcGuigan/ai-games/tree/master/puzzles/game_of_life

In [ ]:
%load_ext autoreload
%autoreload 2

import itertools
import time
import numpy as np
import pandas as pd
import pydash
from typing import Union, List, Tuple
import os
import sys
from pathos.multiprocessing import ProcessPool

from utils.plot import *
from utils.game import *
from utils.util import *
from utils.datasets import *
from utils.display_source import *

print('os.cpu_count()', os.cpu_count())
notebook_start = time.perf_counter()

In [ ]:
# These imports won't work inside Kaggle Submit without an internet connection to install Z3
import z3
from constraint_satisfaction.z3_solver import *
from constraint_satisfaction.solve_dataframe import *

This is the main codebase implementing the Z3 SAT Solver.

- using Boolean rather than Integer logic results in a 2-4x speedup in solve times. 
- adding an additional constraint that any dead cell with zero neighbours should also be considered dead in the T=-1 timeframe dramatially reduces the state search space and prevents zero-point energy solutions (cells that are born and die in the vaccume of whitespace)

In [ ]:
display_source('./constraint_satisfaction/z3_solver.py')

In [ ]:
display_source('./constraint_satisfaction/z3_constraints.py')

This is the main multiprocessing loop for processing submission.csv

In [ ]:
display_source('./constraint_satisfaction/solve_dataframe.py')

In [ ]:
display_source('./utils/idx_lookup.py')

This is the forward implemention of the game of life implemented 3 different ways, with profiler measurements
```
  42.7µs - lambda: [ life_step(x)    for x in dataset ],  # 2882.0µs without numba
 200.1µs - lambda: [ life_step_1(x)  for x in dataset ],
  38.7µs - lambda: [ life_step_2(x)  for x in dataset ],
```

In [ ]:
display_source('./utils/game.py')

Casting functions for converting between datatypes

In [ ]:
display_source('./utils/util.py')

This lets us plot out user-friendly visualiztions

In [ ]:
display_source('./utils/plot.py')

Deployment specific lookups for the dataset .csv files 

In [ ]:
display_source('./utils/datasets.py')

Test that all answers in the in the submission.csv file, when played forward, match the input data

In [ ]:
display_source('./test/test_submission.py')

# Dataset

In [ ]:
test_df

# Range of Possible Solutions   

Here we show the first 10 solutions to the first (very simple) datapoint, having excluded most of the zero-point energy solutions (ie clusters of cells that appear and dissapear in the vaccume of whitespace).

To find the exact starting conditions used by kaggle, would require computing valid solutions to T=-5 to account for the warmup period, but this can be very slow.

Note how for even very simple starting conditions, there is a very large range of possible solutions

In [ ]:
idx      = 0  
delta    = csv_to_delta(train_df, idx)
board    = csv_to_numpy(train_df, idx, key='stop')
expected = csv_to_numpy(train_df, idx, key='start')

time_start     = time.perf_counter()
solution_count = 0
z3_solver, t_cells, solution_3d = game_of_life_solver(board, delta, idx=idx)
while np.count_nonzero(solution_3d):
    solution_count += 1
    plot_3d(solution_3d)
    z3_solver, t_cells, solution_3d = game_of_life_next_solution(z3_solver, t_cells, verbose=True) # takes ~0.5s per solution
    if solution_count > 5: break
# print(f'Total Solutions: {solution_count} in {time.perf_counter() - time_start:.1f}s')  # too many to count

# Train Dataset

In [ ]:
idx      = 0
delta    = csv_to_delta(train_df, idx)
board    = csv_to_numpy(train_df, idx, key='stop')
solution_3d, idx, time_taken = solve_board_delta1_loop(board, delta, idx)
plot_3d( solution_3d )

# Test Dataset

As we can see, large deltas are exponentially more difficult to solve than smaller deltas.

We can also use either joblib or pathos.multiprocessing to make full use of the 4 vCPU cores available in the Kaggle Kernel. 

The modulo paramter lets me split training between kaggle notebooks and localhost compute whilst retaining sorting (lets solve the easy ones first). 

In [ ]:
# test_df.loc[72539] | delta = 1 | time = 10.4s
# test_df.loc[56795] | delta = 1 | time = 9.5s
# test_df.loc[58109] | delta = 2 | time = 21.1s
# test_df.loc[62386] | delta = 2 | time = 20.5s
# test_df.loc[64934] | delta = 3 | time = 41.4s
# test_df.loc[77908] | delta = 3 | time = 49.5s
# test_df.loc[55567] | delta = 4 | time = 151.8s
# test_df.loc[71076] | delta = 4 | time = 239.0s
# test_df.loc[74622] | delta = 5 | time = 2119.0s
# test_df.loc[75766] | delta = 5 | time = 1518.6s

import random
from joblib import delayed
from joblib import Parallel
submision_df = pd.read_csv('submission.csv', index_col='id')  # manually copy/paste sample_submission.csv to location

job_idxs = []
delta_batch_indexes = [0,10,100] 
delta_idxs = {}
for delta in [1,2,3,4,5]:  # sorted(test_df['delta'].unique()):  # [1,2,3,4,5]
    df   = test_df
    df   = df[ df['delta'] == delta ]                                 # smaller deltas are exponentially easier
    df   = df.iloc[ df.apply(np.count_nonzero, axis=1).argsort()  ]   # smaller grids are easiest 
    # idxs = get_unsolved_idxs(df, submision_df, modulo=(10,1) )      # don't recompute solved boards
    idxs = list(df.index)                                             # show solved small boards
    idxs = idxs[delta_batch_indexes]                                  # sample of small boards 
    delta_idxs[delta] = idxs
    # quartiles from each delta - this takes far too long
    # idxs = [ idxs[0], idxs[len(idxs)*1//4], idxs[len(idxs)*1//2], idxs[len(idxs)*3//4], idxs[-1] ] 
    job_idxs += idxs
print('delta_idxs', delta_idxs)
    
jobs = []    
df   = test_df
for n, idx in enumerate(job_idxs):
    delta = csv_to_delta(df, idx)
    board = csv_to_numpy(df, idx, key='stop')
    def job_fn(board, delta, idx):
        time_start = time.perf_counter()
        z3_solver, t_cells, solution_3d = game_of_life_solver(board, delta, idx=idx, verbose=True)
        time_taken = time.perf_counter() - time_start
        return solution_3d, idx, delta, time_taken 
    jobs.append( delayed(job_fn)(board, delta, idx) )
        
# jobs = []       
# pathos.multiprocessing: Pool.uimap() is used for the submission.csv loop as it uses iterator rather than batch semantics  
jobs_output = Parallel(-2)(reversed(jobs))      # run longest jobs first 
for solution_3d, idx, delta, time_taken in reversed(jobs_output):
    print(f'test_df.loc[{idx}] | delta = {delta} | cells = {np.count_nonzero(solution_3d[-1])} | time = {time_taken:.1f}s')
    if is_valid_solution_3d(solution_3d):        
        plot_3d(solution_3d)
        # Save to submission.csv.csv file         
        submision_df          = pd.read_csv('submission.csv', index_col='id')
        solution_dict         = numpy_to_dict(solution_3d[0])
        submision_df.loc[idx] = pd.Series(solution_dict)
        submision_df.sort_index().to_csv('submission.csv')
    else:
        print('Unsolveable!')
        plot_idx(test_df, idx)

# Submission

There are 50,000 data points in the test dataset, which divided by the 9 hour notebook timeout gives us a performance requirement of 0.6s to solve each datapoint.

For performance optimiztion, `solve_dataframe()` uses pathos.multiprocessing, uses submission.csv as a cache file and persists after each solution is found, avoiding recomputing non-zero entries already in the file. Kaggle provides us with upto 10 * 9 hour commit sessions at once. `modulo % 10 == N` is used to evenly divide the dataset between 10 forked copies of itself, that can all be run simultaneously as the poor man's version of cluster compute. The notebooks then all reimport each other's output files and the multiple .csv files are merged in bash using `find|xargs|sort|awk`

Even still, 50,000 entries is going to require a very large amount of compute, with notebooks being manually restarted every 9 hours. All I need to do is to [keep pushing the button](https://www.youtube.com/watch?v=xsksWR8uTDQ&ab_channel=busterroni11) without getting Lost.

In [ ]:
notebook_time = (time.perf_counter() - notebook_start)
print(f'notebook_time = {notebook_time:.0f}s = {notebook_time/60:.1f}m')

In [ ]:
# submission_df = solve_dataframe(test_df, savefile='submission.csv', timeout=(7*60*60 - notebook_time), modulo=(1,0), max_timeout=3*60*60, plot=True)
# submission_df = solve_dataframe(test_df, savefile='submission.csv', timeout=(0.1*60*60 - notebook_time), modulo=(9,1), plot=True)

In [ ]:
# Cleanup python caches to prevent poluting kaggle notebook outputs
!find ./ -name '__pycache__' -or -name '*.py[cod]'  -delete

# Cluster Progress Counts

As a Count of [Sealand](https://en.wikipedia.org/wiki/Principality_of_Sealand), I do indeed [like to count](https://www.youtube.com/watch?v=ZIniljT5lJI), so this notbook has to end with a count of successfully solved games:

In [ ]:
# BUGFIX: previous version of the code was computing to delta=-1, so replay submission.csv forward one step if required and validate we have the correct delta
# This also generates stats
!PYTHONPATH='.' python3 ./constraint_satisfaction/fix_submission.py

In [ ]:
# Count number of non-zero entries in each submission.csv file
!( for FILE in *.csv; do cat $FILE | grep ',1,' | wc -l | tr '\n' ' '; echo $FILE; done ) | sort -n;

# Further Reading

I have written an interactive playable demo of the forward version of this game in React Javascript:
- https://life.jamesmcguigan.com/


This notebook is part of series exploring the Neural Network implementions of the Game of Life Foward Problem
- [Pytorch Game of Life - First Attempt](https://www.kaggle.com/jamesmcguigan/pytorch-game-of-life-first-attempt)
- [Pytorch Game of Life - Hardcoding Network Weights](https://www.kaggle.com/jamesmcguigan/pytorch-game-of-life-hardcoding-network-weights)
- [Its Easy for Neural Networks To Learn Game of Life](https://www.kaggle.com/jamesmcguigan/its-easy-for-neural-networks-to-learn-game-of-life)

This is preliminary research towards the harder Reverse Game of Life problem, for which I have already designed a novel Ouroboros loss function: 
- [OuroborosLife - Function Reversal GAN](https://www.kaggle.com/jamesmcguigan/ouroboroslife-function-reversal-gan)


I also have an extended series of Notebooks exploring different approaches to the Reverse Game of Life problem

My first attempt was to use the Z3 Constraint Satisfaction SAT solver. This gets 100% accuracy on most boards, but there are a few which it cannot solve. This approach can be slow for boards with large cell counts and large deltas. I managed to figure out how to get cluster compute working inside Kaggle Notebooks, but this solution is estimated to require 10,000+ hours of CPU time to complete.    
- [Game of Life - Z3 Constraint Satisfaction](https://www.kaggle.com/jamesmcguigan/game-of-life-z3-constraint-satisfaction)

Second approach was to create a Geometrically Invarient Hash function using Summable Primes, then use forward play and a dictionary lookup table to create a database of known states. For known input/output states at a given delta, the problem is reduced to simply solving the geometric transform between inputs and applying the same function to the outputs. The Hashmap Solver was able to solve about 10% of the test dataset. 
- [Summable Primes](https://www.kaggle.com/jamesmcguigan/summable-primes)
- [Geometric Invariant Hash Functions](https://www.kaggle.com/jamesmcguigan/geometric-invariant-hash-functions)
- [Game of Life - Repeating Patterns](https://www.kaggle.com/jamesmcguigan/game-of-life-repeating-patterns)
- [Game of Life - Hashmap Solver](https://www.kaggle.com/jamesmcguigan/game-of-life-hashmap-solver)
- [Game of Life - Image Segmentation Solver](https://www.kaggle.com/jamesmcguigan/game-of-life-image-segmentation-solver)